In [8]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

header = 'chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

#데이터 추출
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
feeling = '0 1 2 3 4 5 6 7'.split()
for f in feeling:
    for filename in os.listdir(f'./MIR/{f}'):
        songname = f'./MIR/{f}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {f}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())


Using TensorFlow backend.


NameError: name 'models' is not defined

In [9]:
data = pd.read_csv('data.csv')

scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

encoder = LabelEncoder()
genre_list = data.iloc[:, -1]
y = encoder.fit_transform(genre_list)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train_t = X_train.reshape(X_train.shape[0], 25, 1)
X_test_t = X_test.reshape(X_test.shape[0], 25, 1)

In [11]:
X_train_t.shape, X_test_t.shape

((604, 25, 1), (152, 25, 1))

In [121]:
from keras.layers import LSTM 
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout, Conv1D, LSTM

#모델
X_train_t = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_t = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

x_val = X_train[:200]
y_val = y_train[:200]

model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(25, 1)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)
model.fit(X_train_t, y_train, epochs=300, batch_size=512, verbose=1, validation_data = [X_test_t, y_test])



Train on 604 samples, validate on 152 samples
Epoch 1/300
604/604 [==============================] - 1s 1ms/sample - loss: 2.0798 - accuracy: 0.1391 - val_loss: 2.0750 - val_accuracy: 0.1184
Epoch 2/300
604/604 [==============================] - 0s 201us/sample - loss: 2.0746 - accuracy: 0.1540 - val_loss: 2.0720 - val_accuracy: 0.1250
Epoch 3/300
604/604 [==============================] - 0s 190us/sample - loss: 2.0713 - accuracy: 0.1573 - val_loss: 2.0681 - val_accuracy: 0.1382
Epoch 4/300
604/604 [==============================] - 0s 192us/sample - loss: 2.0671 - accuracy: 0.1738 - val_loss: 2.0636 - val_accuracy: 0.1711
Epoch 5/300
604/604 [==============================] - 0s 185us/sample - loss: 2.0626 - accuracy: 0.2020 - val_loss: 2.0588 - val_accuracy: 0.1908
Epoch 6/300
604/604 [==============================] - 0s 183us/sample - loss: 2.0580 - accuracy: 0.2103 - val_loss: 2.0542 - val_accuracy: 0.1842
Epoch 7/300
604/604 [==============================] - 0s 192us/sample - l

604/604 [==============================] - 0s 183us/sample - loss: 1.5875 - accuracy: 0.3808 - val_loss: 1.7720 - val_accuracy: 0.3026
Epoch 57/300
604/604 [==============================] - 0s 183us/sample - loss: 1.5919 - accuracy: 0.3825 - val_loss: 1.8005 - val_accuracy: 0.3158
Epoch 58/300
604/604 [==============================] - 0s 195us/sample - loss: 1.5771 - accuracy: 0.3907 - val_loss: 1.8314 - val_accuracy: 0.2961
Epoch 59/300
604/604 [==============================] - 0s 198us/sample - loss: 1.5843 - accuracy: 0.3907 - val_loss: 1.7964 - val_accuracy: 0.3026
Epoch 60/300
604/604 [==============================] - 0s 201us/sample - loss: 1.5851 - accuracy: 0.3825 - val_loss: 1.8272 - val_accuracy: 0.3092
Epoch 61/300
604/604 [==============================] - 0s 188us/sample - loss: 1.5828 - accuracy: 0.3825 - val_loss: 1.8443 - val_accuracy: 0.3224
Epoch 62/300
604/604 [==============================] - 0s 183us/sample - loss: 1.5728 - accuracy: 0.3858 - val_loss: 1.7808 

604/604 [==============================] - 0s 193us/sample - loss: 1.5172 - accuracy: 0.4023 - val_loss: 1.7939 - val_accuracy: 0.3158
Epoch 112/300
604/604 [==============================] - 0s 188us/sample - loss: 1.5056 - accuracy: 0.4205 - val_loss: 1.8196 - val_accuracy: 0.2961
Epoch 113/300
604/604 [==============================] - 0s 190us/sample - loss: 1.5051 - accuracy: 0.4205 - val_loss: 1.8827 - val_accuracy: 0.2829
Epoch 114/300
604/604 [==============================] - 0s 201us/sample - loss: 1.5055 - accuracy: 0.4255 - val_loss: 1.9102 - val_accuracy: 0.3026
Epoch 115/300
604/604 [==============================] - 0s 193us/sample - loss: 1.5077 - accuracy: 0.4172 - val_loss: 1.8171 - val_accuracy: 0.3026
Epoch 116/300
604/604 [==============================] - 0s 183us/sample - loss: 1.5031 - accuracy: 0.4073 - val_loss: 1.7944 - val_accuracy: 0.3092
Epoch 117/300
604/604 [==============================] - 0s 198us/sample - loss: 1.5022 - accuracy: 0.4089 - val_loss: 1

604/604 [==============================] - 0s 200us/sample - loss: 1.4475 - accuracy: 0.4172 - val_loss: 1.8707 - val_accuracy: 0.2895
Epoch 167/300
604/604 [==============================] - 0s 203us/sample - loss: 1.4631 - accuracy: 0.4106 - val_loss: 1.8400 - val_accuracy: 0.3289
Epoch 168/300
604/604 [==============================] - 0s 201us/sample - loss: 1.5327 - accuracy: 0.4139 - val_loss: 1.8409 - val_accuracy: 0.3289
Epoch 169/300
604/604 [==============================] - 0s 193us/sample - loss: 1.5564 - accuracy: 0.4106 - val_loss: 1.7815 - val_accuracy: 0.3289
Epoch 170/300
604/604 [==============================] - 0s 187us/sample - loss: 1.5119 - accuracy: 0.4156 - val_loss: 1.8078 - val_accuracy: 0.3158
Epoch 171/300
604/604 [==============================] - 0s 198us/sample - loss: 1.5317 - accuracy: 0.4023 - val_loss: 1.8104 - val_accuracy: 0.2895
Epoch 172/300
604/604 [==============================] - 0s 188us/sample - loss: 1.5150 - accuracy: 0.4073 - val_loss: 1

604/604 [==============================] - 0s 188us/sample - loss: 1.3617 - accuracy: 0.4487 - val_loss: 1.9896 - val_accuracy: 0.2829
Epoch 222/300
604/604 [==============================] - 0s 183us/sample - loss: 1.3499 - accuracy: 0.4553 - val_loss: 2.0286 - val_accuracy: 0.2829
Epoch 223/300
604/604 [==============================] - 0s 183us/sample - loss: 1.3529 - accuracy: 0.4520 - val_loss: 2.0044 - val_accuracy: 0.2632
Epoch 224/300
604/604 [==============================] - 0s 195us/sample - loss: 1.3485 - accuracy: 0.4619 - val_loss: 2.0156 - val_accuracy: 0.2632
Epoch 225/300
604/604 [==============================] - 0s 198us/sample - loss: 1.3510 - accuracy: 0.4652 - val_loss: 2.0630 - val_accuracy: 0.2697
Epoch 226/300
604/604 [==============================] - 0s 190us/sample - loss: 1.3462 - accuracy: 0.4669 - val_loss: 2.0612 - val_accuracy: 0.2697
Epoch 227/300
604/604 [==============================] - 0s 183us/sample - loss: 1.3367 - accuracy: 0.4619 - val_loss: 2

604/604 [==============================] - 0s 187us/sample - loss: 1.2762 - accuracy: 0.4702 - val_loss: 2.3764 - val_accuracy: 0.2632
Epoch 277/300
604/604 [==============================] - 0s 183us/sample - loss: 1.2481 - accuracy: 0.4967 - val_loss: 2.4475 - val_accuracy: 0.2566
Epoch 278/300
604/604 [==============================] - 0s 183us/sample - loss: 1.2484 - accuracy: 0.4983 - val_loss: 2.4743 - val_accuracy: 0.2961
Epoch 279/300
604/604 [==============================] - 0s 187us/sample - loss: 1.2996 - accuracy: 0.4702 - val_loss: 2.5543 - val_accuracy: 0.2500
Epoch 280/300
604/604 [==============================] - 0s 183us/sample - loss: 1.2481 - accuracy: 0.5000 - val_loss: 2.5454 - val_accuracy: 0.2632
Epoch 281/300
604/604 [==============================] - 0s 183us/sample - loss: 1.2488 - accuracy: 0.5050 - val_loss: 2.5004 - val_accuracy: 0.2697
Epoch 282/300
604/604 [==============================] - 0s 187us/sample - loss: 1.2997 - accuracy: 0.4768 - val_loss: 2

In [123]:
results = model.evaluate(X_test_t, y_test)

predictions = model.predict(X_test_t)
predictions[0].shape #(10,)
np.sum(predictions[0])
np.argmax(predictions[0])



152/152 [==============================] - 0s 138us/sample - loss: 2.4336 - accuracy: 0.2566


4

In [124]:
#예측
from keras.models import load_model
import pandas as pd

data = pd.read_csv('./test/testdata.csv')
a = np.array(data)
b = a.reshape(1, data.shape[1], data.shape[0])
b.shape

(1, 25, 1)

In [126]:
predictions = model.predict(b)
predictions

array([[0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [72]:
y, sr = librosa.load("[SongGain] 송가인 - 엄마 아리랑.mp3", mono=True)                

chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)


# to_append = f'{np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
# for e in mfcc:
#     to_append += f' {np.mean(e)}'

# file = open('testdata.csv', 'a', newline='')
# with file:
#     writer = csv.writer(file)
#     writer.writerow(to_append.split())

In [74]:
np.mean(chroma_stft)

0.29456612

In [88]:
np.mean(spec_cent)

2279.4938241315404

In [99]:
chroma_stft.shape, rolloff.shape, spec_cent.shape, spec_bw.shape, zcr.shape

((12, 20446), (1, 20446), (1, 20446), (1, 20446), (1, 20446))

In [106]:
a = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)

In [110]:
b = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr).T, axis=0)

In [111]:
c = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr).T, axis=0)

In [ ]:
행렬의 축을 바꾼다 -> .T
axis = 0 -> x축 기준